In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 1) .env 불러오기
load_dotenv()

# 2) LangChain용 LLM 객체 만들기 (Azure OpenAI v1 방식)
model = ChatOpenAI(
    model=os.getenv("DEPLOYMENT_NAME"),               
    api_key=os.getenv("OPENAI_API_KEY"),            
    base_url=os.getenv("ENDPOINT_URL").rstrip("/") + "/openai/v1/",
    temperature=0.0,
)

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage("잘 지냈어? 지금 현재 시간이 몇시야? ")])

AIMessage(content='안녕하세요! 저는 현재 시간을 직접 알 수 없습니다.  \n제가 사용하는 시스템에서는 실시간 시계 기능이 없어서, 현재 시간을 알려드릴 수 없어요.  \n혹시 다른 도움이 필요하시면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 20, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-Ce56HrNL5hG9WZH3xZnSo0QDGzjak', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--763f3bfa-e53e-4f36-bf4f-b568a606b746-0', usage_metadata={'input_tokens': 20, 'output_tokens': 47, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.tools import tool
from datetime import datetime
import pytz


In [8]:
@tool
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시간을 반환하는 함수
    
    Args:
    timezone (str): 타임존(예: 'Asia/Seoul'). 실제 존재해야 함
    location (str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 modle 답변 생성에 사용됨
    
    """

    tz = pytz.timezone(timezone)

    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now}'
    print(location_and_local_time)
    
    return location_and_local_time

In [9]:
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time, }

model_with_tools = model.bind_tools(tools)

In [10]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자 답변을 하기 위해 Tool을 사용할 거야"),
    HumanMessage("파리는 지금 몇 시야?"),
]

response = model_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='너는 사용자 답변을 하기 위해 Tool을 사용할 거야', additional_kwargs={}, response_metadata={}), HumanMessage(content='파리는 지금 몇 시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 124, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-Ce57L7VeeyloAJUmoWGkjXleamBFc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--3018f7d5-8657-4e57-a193-73e36b90497b-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Europe/Paris', 'location': 'Paris'}, 'id': 'call_vyq8AUKzN3iGZYwkYOM8vYH4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1

In [11]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Europe/Paris', 'location': 'Paris'}
Europe/Paris (Paris) 현재시각 2025-11-20 21:00:30


[SystemMessage(content='너는 사용자 답변을 하기 위해 Tool을 사용할 거야', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='파리는 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 124, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-Ce57L7VeeyloAJUmoWGkjXleamBFc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--3018f7d5-8657-4e57-a193-73e36b90497b-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Europe/Paris', 'location': 'Paris'}, 'id': 'call_vyq8AUKzN3iGZYwkYOM8vYH4', 'type': 'tool_call'}], usage_metadata={'input_tokens':

In [12]:
model_with_tools.invoke(messages)

AIMessage(content='파리의 현재 시각은 2025년 11월 20일 21시 00분 30초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 176, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-Ce57Q5zmztQ6OxZV75sJhVkqfwfbE', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--db92e7eb-b450-49fb-9139-15453a51cfae-0', usage_metadata={'input_tokens': 176, 'output_tokens': 30, 'total_tokens': 206, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
for c in model.stream([HumanMessage("잘 지냈어? 오픈소스SW에 대해서 무엇인지 설명해줘")]):
    print(c.content, end='')

안녕하세요! 잘 지냈어요. 😊  
오픈소스 소프트웨어(Open Source Software, OSS)에 대해 설명해드릴게요.

**오픈소스 소프트웨어란?**  
오픈소스 소프트웨어는 소스 코드가 공개되어 누구나 자유롭게 사용할 수 있고, 수정하거나 배포할 수 있는 소프트웨어를 말합니다. 즉, 소프트웨어의 설계도(소스 코드)가 공개되어 있어서 개발자나 사용자가 자유롭게 접근하고, 필요에 따라 개선하거나 기능을 추가할 수 있습니다.

**주요 특징**
- **소스 코드 공개**: 누구나 소스 코드를 볼 수 있습니다.
- **자유로운 사용**: 개인, 기업 등 누구나 사용할 수 있습니다.
- **수정 및 배포 가능**: 소프트웨어를 수정하거나, 수정한 버전을 다시 배포할 수 있습니다.
- **라이선스**: 오픈소스 소프트웨어는 보통 GPL, MIT, Apache 등과 같은 오픈소스 라이선스를 따릅니다. 각 라이선스마다 사용, 수정, 배포 조건이 다를 수 있습니다.

**대표적인 오픈소스 소프트웨어 예시**
- **리눅스(Linux)**: 대표적인 오픈소스 운영체제
- **아파치(Apache)**: 웹 서버 소프트웨어
- **파이썬(Python)**: 프로그래밍 언어
- **파이어폭스(Firefox)**: 웹 브라우저

**장점**
- 비용 절감(무료로 사용 가능)
- 커뮤니티의 활발한 지원과 빠른 버그 수정
- 다양한 기능 추가 및 커스터마이징 가능

**단점**
- 공식적인 지원이 부족할 수 있음
- 사용 및 배포 시 라이선스 조건을 잘 확인해야 함

궁금한 점이 더 있으면 언제든 질문해 주세요!